In [22]:
import operator
import numpy as np
import pickle, string, re
from pyvi import ViTokenizer, ViPosTagger
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

In [23]:
def normalText(sent):
    sent = str(sent).replace('_',' ').replace('/',' trên ')
    sent = re.sub('-{2,}','',sent)
    sent = re.sub('\\s+',' ', sent)
    patPrice = r'([0-9]+k?(\s?-\s?)[0-9]+\s?(k|K))|([0-9]+(.|,)?[0-9]+\s?(triệu|ngàn|trăm|k|K|))|([0-9]+(.[0-9]+)?Ä‘)|([0-9]+k)'
    patURL = r"(?:http://|www.)[^\"]+"
    sent = re.sub(patURL,'website',sent)
    sent = re.sub(patPrice, ' giá_tiền ', sent)
    sent = re.sub('\.+','.',sent)
    sent = re.sub('(hagtag\\s+)+',' hagtag ',sent)
    sent = re.sub('\\s+',' ',sent)
    return sent

def deleteIcon(text):
    text = text.lower()
    s = ''
    pattern = r"[a-zA-ZaăâbcdđeêghiklmnoôơpqrstuưvxyàằầbcdđèềghìklmnòồờpqrstùừvxỳáắấbcdđéếghíklmnóốớpqrstúứvxýảẳẩbcdđẻểghỉklmnỏổởpqrstủửvxỷạặậbcdđẹệghịklmnọộợpqrstụựvxỵãẵẫbcdđẽễghĩklmnõỗỡpqrstũữvxỹAĂÂBCDĐEÊGHIKLMNOÔƠPQRSTUƯVXYÀẰẦBCDĐÈỀGHÌKLMNÒỒỜPQRSTÙỪVXỲÁẮẤBCDĐÉẾGHÍKLMNÓỐỚPQRSTÚỨVXÝẠẶẬBCDĐẸỆGHỊKLMNỌỘỢPQRSTỤỰVXỴẢẲẨBCDĐẺỂGHỈKLMNỎỔỞPQRSTỦỬVXỶÃẴẪBCDĐẼỄGHĨKLMNÕỖỠPQRSTŨỮVXỸ,._]"
    
    for char in text:
        if char !=' ':
            if len(re.findall(pattern, char)) != 0:
                s+=char
            elif char == '_':
                s+=char
        else:
            s+=char
    s = re.sub('\\s+',' ',s)
    return s.strip()


def clean_doc(doc):
    for punc in string.punctuation:
        doc = doc.replace(punc,' '+ punc + ' ')
    doc = normalText(doc)
    doc = deleteIcon(doc)
    doc = ViTokenizer.tokenize(doc)
    doc = doc.lower()
    doc = re.sub(r"\?", " \? ", doc)
    doc = re.sub(r"[0-9]+", " num ", doc)
    for punc in string.punctuation:
        if punc not in "_":
            doc = doc.replace(punc,' ')
    doc = re.sub('\\s+',' ',doc)
    return doc

In [24]:
path_save ='model/'
model = load_model(path_save + 'ResCNN_model.h5')

with open('model/input_tokenizer.pkl', 'rb') as fp:
    input_tokenizer = pickle.load(fp)

In [40]:
!pip install flask

In [25]:
maxLength = 548

In [69]:
input_text = "Dở như cứt, phục vụ tệ, tính tiền đắt vcl"
input_clean = clean_doc(input_text)
input_index = np.array(pad_sequences(input_tokenizer.texts_to_sequences([input_clean]), maxlen=maxLength,padding="post"))

listLabel = 'FOOD#STYLE&OPTIONS,FOOD#PRICES,FOOD#QUALITY,DRINKS#STYLE&OPTIONS,DRINKS#QUALITY,DRINKS#PRICES,RESTAURANT#GENERAL,RESTAURANT#MISCELLANEOUS,RESTAURANT#PRICES,LOCATION#GENERAL,SERVICE#GENERAL,AMBIENCE#GENERAL'
categories = listLabel.split(',')

predicted = model.predict([np.expand_dims(input_index[0], axis=0)])
output = {'tag':[], 'value':[]}
for i, predict in enumerate(predicted):
    index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
    if index2 == 1:
        output['tag'].append(str(categories[i]))
        output['value'].append("positive")
        #output[str(categories[i])] = "positive"
    elif index2 == 2:
        output['tag'].append(str(categories[i]))
        output['value'].append("neutral")
        #output[str(categories[i])] = "neutral"
    elif index2 == 3:
        output['tag'].append(str(categories[i]))
        output['value'].append("negative")
        #output[str(categories[i])] = "negative"
print("Đầu vào: ", input_text)
print("Dự đoán: ", output)

Đầu vào:  Dở như cứt, phục vụ tệ, tính tiền đắt vcl
Dự đoán:  {'tag': ['FOOD#PRICES', 'FOOD#QUALITY', 'SERVICE#GENERAL'], 'value': ['negative', 'neutral', 'negative']}


In [70]:
from flask import Flask, request, jsonify

In [76]:
app = Flask(__name__)
@app.route('/setmodel', methods=['POST'])
def a():
    return jsonify(output)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [77]:
import json
out = json.dumps(output)

In [78]:
print(output['tag'])

['FOOD#PRICES', 'FOOD#QUALITY', 'SERVICE#GENERAL']


In [80]:
print(type(out))

<class 'str'>
